In [1]:
import pandas as pd
import numpy as np
import datetime

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

# Pivot

In [2]:
import pandas._testing as tm

tm.N = 3


def unpivot(frame):
    N, K = frame.shape
    data = {'value': frame.to_numpy().ravel('F'),
            'variable': np.asarray(frame.columns).repeat(N),
            'date': np.tile(np.asarray(frame.index), K)}
    return pd.DataFrame(data, columns=['date', 'variable', 'value'])


df = unpivot(tm.makeTimeDataFrame())
df

,date,variable,value
0,2000-01-03,A,0.212630
1,2000-01-04,A,0.149708
2,2000-01-05,A,-0.371661
3,2000-01-03,B,0.011807
4,2000-01-04,B,1.417275
5,2000-01-05,B,-0.066813
6,2000-01-03,C,-0.003797
7,2000-01-04,C,-0.610416
8,2000-01-05,C,-1.113840
9,2000-01-03,D,0.609603


In [3]:
df.pivot(index='date',columns='variable',values='value')

variable,A,B,C,D
date,,,,
2000-01-03,0.212630,0.011807,-0.003797,0.609603
2000-01-04,0.149708,1.417275,-0.610416,-0.218793
2000-01-05,-0.371661,-0.066813,-1.113840,0.388427


In [4]:
df['value2']= df['value']*2
df

,date,variable,value,value2
0,2000-01-03,A,0.212630,0.425260
1,2000-01-04,A,0.149708,0.299416
2,2000-01-05,A,-0.371661,-0.743322
3,2000-01-03,B,0.011807,0.023613
4,2000-01-04,B,1.417275,2.834550
5,2000-01-05,B,-0.066813,-0.133626
6,2000-01-03,C,-0.003797,-0.007594
7,2000-01-04,C,-0.610416,-1.220832
8,2000-01-05,C,-1.113840,-2.227680
9,2000-01-03,D,0.609603,1.219206


In [5]:
df.pivot(index='date',columns='variable')

value                                  value2            \
variable           A         B         C         D         A         B   
date                                                                     
2000-01-03  0.212630  0.011807 -0.003797  0.609603  0.425260  0.023613   
2000-01-04  0.149708  1.417275 -0.610416 -0.218793  0.299416  2.834550   
2000-01-05 -0.371661 -0.066813 -1.113840  0.388427 -0.743322 -0.133626   

                                
variable           C         D  
date                            
2000-01-03 -0.007594  1.219206  
2000-01-04 -1.220832 -0.437586  
2000-01-05 -2.227680  0.776854

# Stack/Unstack

In [6]:
tuples = list(zip(*[['bar', 'bar', 'baz', 'baz','foo', 'foo', 'qux', 'qux'],
                    ['one', 'two', 'one', 'two','one', 'two', 'one', 'two']]))
index=pd.MultiIndex.from_tuples(tuples,names=['first','second'])
df=pd.DataFrame(np.random.randn(8,2),index=index,columns=['A','B'])
df2=df[:4]
display('df','df2')

df
                     A         B
first second                    
bar   one     0.267980 -0.486475
      two     0.658605  0.883719
baz   one     1.126933 -0.735290
      two    -0.537452 -0.225549
foo   one    -0.241117 -0.662890
      two     0.109155  0.742468
qux   one     0.004410 -0.307873
      two    -0.984457 -1.167589

df2
                     A         B
first second                    
bar   one     0.267980 -0.486475
      two     0.658605  0.883719
baz   one     1.126933 -0.735290
      two    -0.537452 -0.225549

In [7]:
df2.stack()

first  second   
bar    one     A    0.267980
               B   -0.486475
       two     A    0.658605
               B    0.883719
baz    one     A    1.126933
               B   -0.735290
       two     A   -0.537452
               B   -0.225549
dtype: float64

###### use stack/unstack with level name

In [8]:
df.unstack('second')

A                   B          
second       one       two       one       two
first                                         
bar     0.267980  0.658605 -0.486475  0.883719
baz     1.126933 -0.537452 -0.735290 -0.225549
foo    -0.241117  0.109155 -0.662890  0.742468
qux     0.004410 -0.984457 -0.307873 -1.167589

###### levels can have multiple levels or names

In [9]:
df.unstack(level=['second','first'])

   second  first
A  one     bar      0.267980
   two     bar      0.658605
   one     baz      1.126933
   two     baz     -0.537452
   one     foo     -0.241117
   two     foo      0.109155
   one     qux      0.004410
   two     qux     -0.984457
B  one     bar     -0.486475
   two     bar      0.883719
   one     baz     -0.735290
   two     baz     -0.225549
   one     foo     -0.662890
   two     foo      0.742468
   one     qux     -0.307873
   two     qux     -1.167589
dtype: float64

In [10]:
df.drop(index=('foo','two'),inplace=True)

###### missing values can be filled it

In [11]:
df.unstack(level=1,fill_value=5)

A                   B          
second       one       two       one       two
first                                         
bar     0.267980  0.658605 -0.486475  0.883719
baz     1.126933 -0.537452 -0.735290 -0.225549
foo    -0.241117  5.000000 -0.662890  5.000000
qux     0.004410 -0.984457 -0.307873 -1.167589

# Melt

In [12]:
cheese = pd.DataFrame({'first': ['John', 'Mary'],
'last': ['Doe', 'Bo'],
'height': [5.5, 6.0],
'weight': [130, 150]})
cheese

,first,last,height,weight
0,John,Doe,5.5,130
1,Mary,Bo,6.0,150


In [13]:
cheese.melt(id_vars=['first','last'],value_name='quanity',var_name='measurement')

,first,last,measurement,quanity
0,John,Doe,height,5.5
1,Mary,Bo,height,6.0
2,John,Doe,weight,130.0
3,Mary,Bo,weight,150.0


# Pivot Tables

In [14]:
df = pd.DataFrame({'A': ['one', 'one', 'two', 'three'] * 6,
'B': ['A', 'B', 'C'] * 8,
'C': ['foo', 'foo', 'foo', 'bar', 'bar', 'bar'] * 4,
'D': np.random.randn(24),
'E': np.random.randn(24),
'F': [datetime.datetime(2013, i, 1) for i in range(1, 13)]
+ [datetime.datetime(2013, i, 15) for i in range(1, 13)]})
df.head()

,A,B,C,D,E,F
0,one,A,foo,-0.714285,0.107980,2013-01-01
1,one,B,foo,0.193363,-1.159710,2013-02-01
2,two,C,foo,-0.111993,-0.596531,2013-03-01
3,three,A,bar,-2.137475,0.682926,2013-04-01
4,one,B,bar,0.326886,1.367569,2013-05-01


In [15]:
df.pivot_table(values='D',index=['A','B'], columns=['C'])

C             bar       foo
A     B                    
one   A  0.236559 -0.301167
      B -0.239602  0.484811
      C  0.084957  0.674225
three A -1.127036       NaN
      B       NaN  0.107500
      C -0.461760       NaN
two   A       NaN -0.063934
      B -0.883140       NaN
      C       NaN -0.972520

In [16]:
df.pivot_table(values='D',index='B',columns=['A','C'], aggfunc=np.sum)

A       one               three                 two          
C       bar       foo       bar       foo       bar       foo
B                                                            
A  0.473118 -0.602334 -2.254072       NaN       NaN -0.127867
B -0.479204  0.969622       NaN  0.215001 -1.766279       NaN
C  0.169913  1.348450 -0.923520       NaN       NaN -1.945040

if agg function contains a list of functions, they will be applied to selected columns.  If agg is a dict with key of col name and value of desired function, only those functions will be applied

###### with dict first

In [17]:
df.pivot_table(values=['D','E'],index='B',columns=['A','C'], 
               aggfunc={'D':np.mean,'E':np.sum})

D                                                        E  \
A       one               three              two                 one   
C       bar       foo       bar     foo      bar       foo       bar   
B                                                                      
A  0.236559 -0.301167 -1.127036     NaN      NaN -0.063934  1.409295   
B -0.239602  0.484811       NaN  0.1075 -0.88314       NaN  1.152927   
C  0.084957  0.674225 -0.461760     NaN      NaN -0.972520 -1.560904   

                                                     
A               three                 two            
C       foo       bar       foo       bar       foo  
B                                                    
A  0.921911 -0.062622       NaN       NaN -1.891149  
B -0.720942       NaN  0.493098 -2.166226       NaN  
C -2.419379 -0.030780       NaN       NaN -1.036132

###### with list

In [18]:
df.pivot_table(values=['D','E'],index='B',columns=['A','C'], 
               aggfunc=[np.mean,np.sum])

mean                                                           \
          D                                                        E   
A       one               three              two                 one   
C       bar       foo       bar     foo      bar       foo       bar   
B                                                                      
A  0.236559 -0.301167 -1.127036     NaN      NaN -0.063934  0.704647   
B -0.239602  0.484811       NaN  0.1075 -0.88314       NaN  0.576464   
C  0.084957  0.674225 -0.461760     NaN      NaN -0.972520 -0.780452   

                                 ...       sum                                \
                                 ...         D                                 
A               three            ...     three                 two             
C       foo       bar       foo  ...       bar       foo       bar       foo   
B                                ...                                           
A  0.460955 -0.031311       NaN  ... -2.254072       NaN       NaN -0.127867   
B -0.360471       NaN  0.246549  ...       NaN  0.215001 -1.766279       NaN   
C -1.209689 -0.015390       NaN  ... -0.923520       NaN       NaN -1.945040   

                                                               
          E                                                    
A       one               three                 two            
C       bar       foo       bar       foo       bar       foo  
B                                                              
A  1.409295  0.921911 -0.062622       NaN       NaN -1.891149  
B  1.152927 -0.720942       NaN  0.493098 -2.166226       NaN  
C -1.560904 -2.419379 -0.030780       NaN       NaN -1.036132  

[3 rows x 24 columns]